# DECIMA2 EXPLANATIONS TAKE HALF THE TIME AS SHAP

In this notebook we show how Decima2 computes model explanations in half the time as the SHAP library on the Adult dataset.

First import the relevant libraries, including decima2.explanation.explanations.model_explanations from the decima2 library 


In [1]:
import shap
from decima2.explanation.explanations import model_explanations
from decima2.utils.utils import feature_names
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import time

Then, we train a Random Forest Classifier on X_train and y_train and evaluate on X_test, y_test

In [2]:
X, y = shap.datasets.adult(n_points=1000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
model = RandomForestClassifier(max_depth=100, random_state=42)
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.855

### Decima2 Model Summary Explanations
We then pass X_test, y_test and our trained Random Forest Classifier to the Decima2 model_explanations function, specifying we only need the 'text' output

In [3]:
st = time.time()
explanation_app = model_explanations(X_test,y_test,model,output='text')
et = time.time()
print("time Decima2 takes to compute is "+str(et-st)+"s")

explanation_app

/Users/tortysivill/.pyenv/versions/test_mvp/lib/python3.12/site-packages/decima2/utils/data_utils.py:202: UserWarning: Increase the number of test instances to increase the reliability of feature importances 
  warnings.warn("Increase the number of test instances to increase the reliability of feature importances ", UserWarning)


{'Workclass': 0.30499999999999994, 'Marital Status': 0.255, 'Relationship': 0.29499999999999993, 'Race': 0.27, 'Sex': 0.19999999999999996, 'Age': 0.22999999999999998, 'Education-Num': 0.22499999999999998, 'Occupation': 0.29999999999999993, 'Capital Gain': 0.245, 'Capital Loss': 0.245, 'Hours per week': 0.19999999999999996, 'Country': 0.245}
time Decima2 takes to compute is 1.4040729999542236s


,Feature,Importance
0,Age,0.305
7,Sex,0.300
2,Education-Num,0.295
3,Marital Status,0.270
1,Workclass,0.255
8,Capital Gain,0.245
9,Capital Loss,0.245
11,Country,0.245
5,Relationship,0.230
6,Race,0.225


### Decima2 Model Summary Explanations
We then pass X_test our trained Random Forest Classifier to the Shap explanations function. SHAP takes double the time as Decima2 explanations!

In [4]:

st = time.time()
explainer = shap.Explainer(model, X_test)
shap_values = explainer(X_test,check_additivity=False)
et = time.time()
print("time SHAP takes to compute is "+str(et-st)+"s")
attributions = shap_values.values.mean(axis=0)[:,0]
attributions = attributions.reshape(X_test.shape[1])
feature_names(X_test,attributions)

time SHAP takes to compute is 3.3597750663757324s


,Feature,Importance
5,Relationship,0.00631
8,Capital Gain,0.00495
0,Age,0.00405
3,Marital Status,0.00217
2,Education-Num,0.00126
10,Hours per week,0.00110
9,Capital Loss,0.00104
11,Country,0.00080
1,Workclass,0.00069
6,Race,0.00057


We can see that our Decima2 explanations run in half the time as the SHAP algorithm!